In [1]:
import collections
import copy
import json
import numpy
import pandas as pd
import pickle
import random
import re
import torch
import tqdm

import padl

Get data for this notebook

In [2]:
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/sentiment-analysis-is-bad/data/training.1600000.processed.noemoticon.csv.zip
!unzip -f training.1600000.processed.noemoticon.csv.zip
!rm training.1600000.processed.noemoticon.csv.zip

--2022-02-18 17:05:16--  https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/sentiment-analysis-is-bad/data/training.1600000.processed.noemoticon.csv.zip
Resolving nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)... 162.243.189.2
Connecting to nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)|162.243.189.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85088192 (81M) [application/zip]
Saving to: ‘training.1600000.processed.noemoticon.csv.zip’

training.1600000.pr 100%[===================>]  81,15M  14,3MB/s    in 6,1s    

2022-02-18 17:05:22 (13,2 MB/s) - ‘training.1600000.processed.noemoticon.csv.zip’ saved [85088192/85088192]

Archive:  training.1600000.processed.noemoticon.csv.zip


Preprocessing for this notebook

In [3]:
df = pd.read_csv(
    'training.1600000.processed.noemoticon.csv',
    header=None,
    encoding='latin-1',
)
    
    
X = df.iloc[:, 5].tolist()
Y = df.iloc[:, 0].tolist()
Y = [{0: 'BAD', 4: 'GOOD'}[y] for y in Y]
    
perm = [int(i) for i in numpy.random.permutation(len(X))]
X_train = [X[i] for i in perm[:-500]]
X_valid = [X[i] for i in perm[-500:]]
Y_train = [Y[i] for i in perm[:-500]]
Y_valid = [Y[i] for i in perm[-500:]]

## Sharing your prototyped results in PyTorch can be tricky

You’ve spent a few days with sweat and tears putting together a PyTorch model in a Jupyter notebook, tinkering with parameters, trying various preprocessing methods, post-processing methods, validating the model results on examples, and checking on validation data sets that the model performs well. The results are good, you’re happy, you’re boss and colleagues are happy. Now you’d like to share the results, so that other people can play with the model; so what do you do? Here are the options:

1. EITHER: Share the notebook
2. AND/ OR: Share the model weights/ JIT compiled model
3. AND/ OR: Reprogram everything in proper scripts so that the model can be reloaded by re-importing functions you created out of the individual notebook cells.

None of these are ideal. Let’s see why:

1. Sharing the notebook only, means that you’d be saving yourself any extra work. But it also means that whoever gets the notebook needs to run it again (reloading the data, retraining the model) in order to get the same results. This may be expensive, or impossible, since you may not be allowed to share the training data.
2. The weights or JIT compiled model are no good by themselves; whoever receives these would need to dissect the notebook in order to make sure that they are putting the tensors into the network(s), and post-processing the outputs in the same way.
3. Reprogramming is a lengthy and error prone process especially for complex workflows/ trainings. At the end you’d hope to have resurrected the exact algorithms/ routines you have in your Jupyter cells, but with tidy signatures, one of which allows the weights which you’d trained to be reloaded, reproducing exactly the results of the notebook. Unfortunately, when you’re finished, most likely the changes are so sweeping that you’re not sure the results are properly replicable based on the refactoring. So you’d then want to run the training one or more times, to verify that the results are the same. In addition, you’ll lose the user friendly/ interactive aspect which the notebook has.

*It doesn’t have to be this way...*

## PADL is a tool which boosts collaboration

[PADL](https://padl.lf1.io/) allows you to work interactively in notebooks, using global variables, inline functions, preprocessing and post-processing, which utilizes the full gamut of the scientific python ecosystem and beyond. When the notebook is done, and you’re happy with the results, you can simply save the pipeline with the PADL serializer. The saved pipeline will exactly replicate the result obtained in the notebook, including the preprocessing and post-processing, including any additional artifacts necessary, data blobs, third party models (such as scikit-learn) and more. This artifact may then be shared, forwarded, experimented with, served and tested in complete isolation from the original notebook. Creating the workflow with PADL, also has a multitude of user friendly additional benefits - transparency, easy debugging, less boilerplate, code which is close to a common graphical mental model for deep learning.

How does this work?

PADL tracks down the code and data necessary to save a full model pipeline using two handy abstractions: the “transform” and the “pipeline”. 

## The "transform" is a basic block of computation

The “transform” is a computational block subsuming preprocessing or postprocessing step, forward pass step or layer into one single class of object. These transforms may be written in a variety of ways. Here are some examples:

In [4]:
# transform tracks code dependencies
@padl.transform                                               
def clean(text):
    return re.sub('[^A-Za-z\ ]', ' ', text)

# transform can wrap a function like this too
split_strip = padl.transform(lambda x: x.strip().split())     

# same allows easy referring to input - like a simple inline lambda
lower_case = padl.same.lower()                                

# callable classes work too!
@padl.transform                                               
class Dictionary:
    def __init__(self, d, default='<unk>'):
        self.d = d
        self.default = default
        
    def __call__(self, token):
        if token in self.d:
            return self.d[token]
        else:
            return self.d[self.default]
        

def save_dictionary(val, path):
    with open(path, 'w') as f:
        json.dump(val, f)
        
        
def load_dictionary(path):
    with open(path) as f:
        return json.load(f)

## Transforms may be linked together into "pipelines"

Once you’ve defined a collection of transforms using `transform`, they may be linked into pipelines using a few primitive operators, leading to a DAG structure for the pipeline. The operators are:

`Map` is the classical well known functional primitive and has short hand `~`. 

In [5]:
(~ clean)(['Testing transform' for _ in range(5)])

('Testing transform',
 'Testing transform',
 'Testing transform',
 'Testing transform',
 'Testing transform')

`Compose` which has the overloading short-hand `>>`. This is similar to composing in, for example, `torchvision`. Transforms or other pipelines’ outputs are passed positionally onto the subsequent objects in the composition. Here’s an example:

In [6]:
text_process = (
    clean
    >> lower_case
    >> split_strip
)

text_process

Compose - "text_process":

      │
      ▼ text
   0: clean                      
      │
      ▼ args
   1: lower()                    
      │
      ▼ x
   2: lambda x: x.strip().split()

## Data artifacts/ blobs can be included in the pipeline

We can use this text processor, for instance, to create the dictionary.

In [7]:
words = []
for sentence in tqdm.tqdm(X_train):
    words.extend(text_process(sentence))
counts = dict(collections.Counter(words))
allowed = sorted(list(counts.keys()), key=lambda x: -counts[x])[:20000]
allowed.append('<unk>')
allowed = padl.value(allowed)
dictionary = Dictionary({k: i for i, k in enumerate(allowed)})
dictionary

100%|██████████| 1599500/1599500 [00:08<00:00, 193479.39it/s]


Dictionary(d={'<unk>': 20000, 'a': 3, 'aa': 4780, 'aaa': 7944, ...}) - "dictionary":

   class Dictionary:
       def __init__(self, d, default='<unk>'):
           self.d = d
           self.default = default

       def __call__(self, token):
           if token in self.d:
               return self.d[token]
           else:
               return self.d[self.default]

You'll notice the use of `padl.value` here - this keyword allows PADL to track data artifacts,
which should also be saved with the pipeline as data blobs (not the code which created them),

Let's add this to the text-processor:

In [8]:
text_process = text_process >> ~ dictionary
text_process

Compose - "text_process":

      │
      ▼ text
   0: clean                                                                 
      │
      ▼ args
   1: lower()                                                               
      │
      ▼ x
   2: lambda x: x.strip().split()                                           
      │
      ▼ args
   3: ~ Dictionary(d={'<unk>': 20000, 'a': 3, 'aa': 4780, 'aaa': 7944, ...})

`Parallel` which has the overloading short-hand `/`. This refers to the situation where multiple transforms are applied “in parallel” to a tuple/ list of outputs from a previous step. This allows you to create complex branching pipelines, providing great flexibility and creativity.

In [9]:
# Parallel sends each part of a tuple to the ith transform in "parallel"
(clean / lower_case)(('Test another&*$', 'Test thingy'))

namedtuple(clean='Test another   ', lower_case='test thingy')

`Rollout` is related to `Parallel` and has the short hand `+`; several transforms are applied over the same input.

In [10]:
(clean + lower_case)('Testing transform')

namedtuple(clean='Testing transform', lower_case='testing transform')

## PyTorch layers may be included organically in your pipeline

PyTorch layers are first class objects in PADL, that means we can decorate the layers directly:

In [11]:
@padl.transform
class TextModel(torch.nn.Module):
    def __init__(self, n_tokens, hidden_size, emb_dim):
        super().__init__()
        self.rnn = torch.nn.GRU(emb_dim, hidden_size=hidden_size,
                                batch_first=True)
        self.embed = torch.nn.Embedding(n_tokens, emb_dim)
        self.output = torch.nn.Linear(hidden_size, 1)
    
    def forward(self, x, lens):
        hidden = self.rnn(self.embed(x))[0]
        last = torch.stack([hidden[i, lens[i] - 1, :]
                            for i in range(hidden.shape[0])])
        return self.output(last)
    
    
layer = TextModel(len(dictionary.d), 1024, 64)

print(layer)

print(isinstance(layer, torch.nn.Module))
print(isinstance(layer, padl.transforms.Transform))

TextModel(n_tokens=20001, hidden_size=1024, emb_dim=64)
True
True


Let’s now create the entire pipeline for classication.

In [12]:
UNK = dictionary('<unk>')
MIN_LEN = 100

@padl.transform
def pad(x):
    return list(x) + [UNK for _ in range(MIN_LEN - len(x))]


@padl.transform
def truncate(x):
    return x[:MIN_LEN]

to_tensor = padl.transform(lambda x: torch.tensor(x))

model = (
    text_process
    >> truncate
    >> pad + padl.transform(lambda x: len(x))
    >> to_tensor / to_tensor
    >> padl.batch
    >> layer
    >> padl.transform(torch.nn.Sigmoid())
    >> padl.same[:, 0]
    >> padl.unbatch
    >> padl.transform(
        lambda x: {False: 'BAD', True: 'GOOD'}[(x > 0.5).item()]
    )
)
model

Compose - "model":

      │
      ▼ text
   0: clean                                                   
      │
      ▼ args
   1: lower()                                                 
      │
      ▼ x
   2: lambda x: x.strip().split()                             
      │
      ▼ args
   3: ~ Dictionary(d={'<unk>': 20000, 'a': 3, 'aa': 4780, 'aa 
      │
      ▼ x
   4: truncate                                                
      └──────────────────────────────────────────────────────────┐
      │                                                          │
      ▼ x                                                        ▼ x
   5: pad                                                      + lambda x: len(x)         
      │                                                          │
      ▼ x                                                        ▼ x
   6: lambda x: torch.tensor(x)                                / lambda x: torch.tensor(x)
      │
      ▼ args
   7: Batchify(dim=0)   

What does all this mean? 

You can see we've added a few more steps to the processing - padding, converting to tensors. These are necessary so that data loading goes through.

The `batch` function is used to automatically construct a data loader. The processing up to `batch` is mapped over the input using multiprocessing.

Between `batch` and `unbatch` is carried out on the GPU, and the part after `unbatch` is performed in serial on the CPU.

This means we save on boilerplate code for data loading, and the whole workflow from raw data to human readable/ useable outputs are together in the pipeline. This can have major practical advantages, such as portability, 
easy debugging, easy model interrogation, collaboration and more.

## PADL allows for iterating through data in several ways

There are 3 ways to pass data through a pipeline with batch - these ways are “infer”, “train”, “eval”.

“Infer”: in this mode, single data points are passed through the model a single batch (batch with one data point) is created and passed to the forward pass.

In [13]:
model.infer_apply('This film was terrible.')

'BAD'

“Eval”: in this mode, data is loaded using multiprocessing and gradients are switched off.

In [14]:
for output in model.eval_apply(['This film was terrible.',
                                'This film was great.'] * 10, batch_size=2):
    print(output)

BAD
BAD
BAD
BAD
BAD
BAD
BAD
BAD
BAD
BAD
BAD
BAD
BAD
BAD
BAD
BAD
BAD
BAD
BAD
BAD


“Train”: in this mode, data is loaded using multiprocessing and gradients are switched on. Here we use the keywords `model.pd_preprocess` (extracts pipeline up to `batch`) and `model.pd_forward` (forward pass part of pipeline).

In [15]:
tensor_outputs = model.pd_preprocess >> model.pd_forward
for output in tensor_outputs.train_apply(['This film was terrible.', 'This film was great.'] * 10, batch_size=2):
    print(output.shape)

torch.Size([2])
torch.Size([2])
torch.Size([2])
torch.Size([2])
torch.Size([2])
torch.Size([2])
torch.Size([2])
torch.Size([2])
torch.Size([2])
torch.Size([2])


## PADL training is completely flexible


Lets get a transform which outputs a loss scalar

In [16]:
targets = (
    padl.transform(lambda x: {'BAD': 0, 'GOOD': 1}[x])
    >> padl.batch
    >> padl.transform(lambda x: x.type(torch.float))
)
loss = tensor_outputs / targets >> padl.transform(torch.nn.BCELoss())

In [17]:
loss

Compose - "loss":

      │└────────────────────────────┐
      │                             │
      ▼ args                        ▼ args
   0: [tensor_outputs: ..>>..] / [targets: ..>>..]
      │
      ▼ (input, target)
   1: BCELoss(torch.nn.BCELoss())

Equipped with these tools, you’re now ready to train the pipeline, which allows for everything you’d expect in a PyTorch training.

In [19]:
o = torch.optim.Adam(model.pd_parameters(), lr=0.0005)
loss.pd_to('cuda')
model.pd_to('cuda')

iteration = 0
try:
    for epoch in range(100):
        for it, l in enumerate(loss.train_apply(list(zip(X_train, Y_train)), batch_size=200, shuffle=True)):
            o.zero_grad()
            l.backward()
            o.step()

            if it % 10 == 0:
                print(f'TRAIN; Epoch: {epoch}; Iteration: {iteration}; Loss: {l}')

            if iteration % 100 == 0:
                predictions = list(model.eval_apply(X_valid, batch_size=200))
                accuracy = sum([a == b for a, b in zip(predictions, Y_valid)]) / len(Y_valid)
                print(f'VALID; Iteration: {iteration}; Epoch: {epoch}; Accuracy: {accuracy}')
            iteration += 1
except KeyboardInterrupt:
    print('quitting...')

TRAIN; Epoch: 0; Iteration: 0; Loss: 0.5170862674713135
VALID; Iteration: 0; Epoch: 0; Accuracy: 0.758
TRAIN; Epoch: 0; Iteration: 10; Loss: 0.45407652854919434
TRAIN; Epoch: 0; Iteration: 20; Loss: 0.4555834233760834
TRAIN; Epoch: 0; Iteration: 30; Loss: 0.4157324433326721
quitting...


## Saving and loading in PADL includes everything

So your model is trained and the results are good! What should you do? Well save it, dear Liza!

In [20]:
padl.save(model, 'mymodel', force_overwrite=True)

saving torch module to mymodel.padl/13.pt
saving torch module to mymodel.padl/14.pt


The following cell works in a completely new session/ after restarting the kernel -- no imports, data processing etc.. required!

In [21]:
reloaded = padl.load('mymodel.padl')
print(reloaded.infer_apply('I am really not very happy right now'))
print(reloaded.infer_apply('I am really stoked to try out this great padl thing!'))

loading torch module from mymodel.padl/13.pt
loading torch module from mymodel.padl/14.pt
BAD
GOOD


## Apply PADL to all your workflows

In this tutorial we implemented an NLP workflow using PyTorch and PADL. However all of this generalizes to the full range of Deep Learning tasks. If you can implement your preprocessing/ forward pass and postprocess with PyTorch and the Python ecosystem, then you can PADL-lize it!

Once your pipeline is in PADL, then you can share the exported pipelines, interact with the steps of the pipelines,
import the pipeline easily, into another session, notebook, or server.

So no more excuses - your model is trained, you are happy, your boss is happy - now your collaborators will be happy too!

**Happy PADL-ling!**